<a href="https://colab.research.google.com/github/PakwhanNK/DP-newsletter-content-analysis/blob/main/03_scraping_newsletter_content.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
!pip install -q beautifulsoup4 lxml requests tqdm

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import re
from google.cloud import bigquery

In [11]:
from google.colab import userdata
PROJECT_ID = userdata.get('PROJECT_ID')
DATASET_ID = userdata.get('DATASET_ID')
client = bigquery.Client(project=PROJECT_ID)


In [13]:
# Get campaigns with archive URLs
query = f"""
SELECT
    id as campaign_id,
    subject_line,
    send_time,
    archive_url,
    long_archive_url
FROM `{PROJECT_ID}.{DATASET_ID}.campaign`
WHERE
    status = 'sent'
    AND archive_url IS NOT NULL
    AND send_time >= '2023-01-01'  -- Last 2 years
ORDER BY send_time DESC
LIMIT 200  -- Start with 200
"""

df_campaigns = client.query(query).to_dataframe()

print(f"✅ Found {len(df_campaigns)} campaigns with URLs")
print(f"\nSample URL: {df_campaigns['archive_url'].iloc[0]}")

df_campaigns.head()

✅ Found 200 campaigns with URLs

Sample URL: http://eepurl.com/jrnaSM


,campaign_id,subject_line,send_time,archive_url,long_archive_url
0,40a96da39f,Your Weekly Toast: On the Boarder–line,2025-11-07 15:00:00+00:00,http://eepurl.com/jrnaSM,https://mailchi.mp/thedp/your-weekly-toast-606...
1,08658dfd18,"Voices of Penn: Mamdani's victory, grade infla...",2025-11-07 13:00:00+00:00,http://eepurl.com/jrolUM,https://mailchi.mp/thedp/voices-of-penn-tktktk...
2,11c93987eb,🏈 Quaker Nation: 2025 Homecoming Preview,2025-11-07 12:00:00+00:00,http://eepurl.com/jrn9FA,https://mailchi.mp/thedp/quaker-nation-april-6...
3,ca0867c19a,Friday Morning: Men’s basketball using NIL col...,2025-11-07 11:00:00+00:00,http://eepurl.com/jrn_vI,https://mailchi.mp/thedp/friday-morning-campus...
4,bb9392f0ac,🏀 Quaker Nation: 2025 Basketball Preview,2025-11-06 12:00:00+00:00,http://eepurl.com/jrhwlw,https://mailchi.mp/thedp/quaker-nation-april-6...


In [14]:
# Test on the most recent newsletter
test_url = df_campaigns['archive_url'].iloc[0]
print(f"🔍 Testing scrape on: {test_url}\n")

# Fetch the HTML
response = requests.get(test_url, timeout=10)
print(f"Status Code: {response.status_code}")

if response.status_code == 200:
    # Parse HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # See the structure
    print("\n📄 HTML Title:", soup.title.string if soup.title else "No title")
    print(f"📄 Total HTML length: {len(response.text):,} characters")

    # Preview raw HTML (first 500 chars)
    print("\n🔍 HTML Preview:")
    print(response.text[:500])
else:
    print(f"❌ Failed to fetch. Status: {response.status_code}")

🔍 Testing scrape on: http://eepurl.com/jrnaSM

Status Code: 200

📄 HTML Title: Your Weekly Toast: On the Boarder–line
📄 Total HTML length: 101,126 characters

🔍 HTML Preview:
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<!doctype html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraph.org/schema/"> <head>
        
<meta property="og:title" content="Your Weekly Toast: On the Boarder–line">
<meta property="fb:page_id" content="43929265776">
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1">
<meta name="referrer" content="origin">        
        <!-- NAME: 1 COL


In [15]:
# Look at the HTML structure to find main content
soup = BeautifulSoup(response.content, 'html.parser')

# Check for common container elements
print("Looking for main content containers...\n")

# Try different selectors
containers = [
    ('div with id="content"', soup.find('div', id='content')),
    ('div with class="content"', soup.find('div', class_='content')),
    ('article tag', soup.find('article')),
    ('main tag', soup.find('main')),
    ('div with id="main"', soup.find('div', id='main')),
    ('div with class="mcnTextContent"', soup.find('div', class_='mcnTextContent')),  # Mailchimp common
]

for name, element in containers:
    if element:
        text = element.get_text(strip=True)
        print(f"✅ {name}: {len(text)} characters")
        print(f"   Preview: {text[:100]}...")
        print()
    else:
        print(f"❌ {name}: Not found")

# Show all div classes (helps identify Mailchimp structure)
print("\n📋 All div classes found:")
all_divs = soup.find_all('div', class_=True)
classes = set()
for div in all_divs[:20]:  # First 20
    classes.update(div.get('class', []))

for cls in sorted(classes):
    print(f"  • {cls}")

🔍 Looking for main content containers...

❌ div with id="content": Not found
❌ div with class="content": Not found
❌ article tag: Not found
❌ main tag: Not found
❌ div with id="main": Not found
❌ div with class="mcnTextContent": Not found

📋 All div classes found:


In [16]:
# Let's see what we're actually dealing with
soup = BeautifulSoup(response.content, 'html.parser')

print("🔍 FULL HTML STRUCTURE ANALYSIS\n")

# 1. Check the title
print(f"📄 Page Title: {soup.title.string if soup.title else 'No title'}")
print()

# 2. Look at all the top-level tags
print("📋 Top-level structure:")
for tag in soup.find_all(recursive=False):
    print(f"  <{tag.name}>")
print()

# 3. Look inside body
if soup.body:
    print("📋 Direct children of <body>:")
    for tag in soup.body.find_all(recursive=False):
        tag_info = f"  <{tag.name}"
        if tag.get('id'):
            tag_info += f' id="{tag.get("id")}"'
        if tag.get('class'):
            tag_info += f' class="{" ".join(tag.get("class"))}"'
        tag_info += ">"
        print(tag_info)
    print()
else:
    print("❌ No body tag found!")

# 4. Find ALL tables (Mailchimp often uses tables for layout)
tables = soup.find_all('table')
print(f"📊 Found {len(tables)} tables in the HTML")
if len(tables) > 0:
    print("\nTable IDs and Classes:")
    for i, table in enumerate(tables[:10]):  # First 10 tables
        table_info = f"  Table {i+1}:"
        if table.get('id'):
            table_info += f' id="{table.get("id")}"'
        if table.get('class'):
            table_info += f' class="{" ".join(table.get("class"))}"'
        # Count text in this table
        text = table.get_text(strip=True)
        table_info += f" ({len(text)} chars)"
        print(table_info)
print()

# 5. Find the table with MOST text (probably the main content!)
print("🎯 Finding table with most content...")
max_text = 0
max_table = None
for table in tables:
    text = table.get_text(strip=True)
    if len(text) > max_text:
        max_text = len(text)
        max_table = table

if max_table:
    print(f"✅ Largest table has {max_text} characters")
    print(f"   Classes: {max_table.get('class', 'None')}")
    print(f"   ID: {max_table.get('id', 'None')}")
    print()
    print("📄 First 300 characters from largest table:")
    print(max_table.get_text(strip=True)[:300])

🔍 FULL HTML STRUCTURE ANALYSIS

📄 Page Title: Your Weekly Toast: On the Boarder–line

📋 Top-level structure:
  <html>

📋 Direct children of <body>:
  <span class="mcnPreviewText">
  <center>
  <script>
  <script>

📊 Found 74 tables in the HTML

Table IDs and Classes:
  Table 1: id="bodyTable" (5501 chars)
  Table 2: (5501 chars)
  Table 3: class="templateContainer" (0 chars)
  Table 4: class="mcnTextBlock" (0 chars)
  Table 5: class="mcnTextContentContainer" (0 chars)
  Table 6: class="templateContainer" (18 chars)
  Table 7: class="mcnImageBlock" (0 chars)
  Table 8: class="mcnImageContentContainer" (0 chars)
  Table 9: class="mcnTextBlock" (18 chars)
  Table 10: class="mcnTextContentContainer" (18 chars)

🎯 Finding table with most content...
✅ Largest table has 5501 characters
   Classes: None
   ID: bodyTable

📄 First 300 characters from largest table:
Friday, November 7Dear Penn,The moral of Avril Lavigne’s “Sk8er Boi” is that even skaters deserve love and respect, right?It’s been 

In [17]:
all_text = soup.get_text(separator=' ', strip=True)

print(f"📊 Total text length: {len(all_text):,} characters")
print(f"📊 Total words: {len(all_text.split()):,}")
print()
print("📄 First 800 characters of ALL text:")
print(all_text[:800])
print()
print("📄 Last 500 characters of ALL text:")
print(all_text[-500:])

📊 Total text length: 5,687 characters
📊 Total words: 969

📄 First 800 characters of ALL text:
Your Weekly Toast: On the Boarder–line This newsletter contains real medical advice from unlicensed psychiatrist Nishanth Bhargava. Friday, November 7 Dear Penn, The moral of Avril Lavigne’s “Sk8er Boi” is that even skaters deserve love and respect, right? It’s been four consecutive days of midterms for me—yes, one per day all this week—with another two outlines due this Friday and my manager poking me about when my deliverables are coming in. The blows just don’t seem to stop coming. I’d say my life has been one battle after another, but that feels disrespectful to Leonardo DiCaprio. After a stressful day, the one thing that helps me unwind is hitting the open road. The moment that highway opens up before me, all my worries melt away. My foot sinks gently into the gas pedal. The sea of ca

📄 Last 500 characters of ALL text:
kicked off this Thursday, but you can catch screenings all through th